## NDVI density classification

Author: Morgane Magnier (morgane.magnier@vattenfall.com)

Copyright © 2024 Magnier Morgane 

This notebook is part of a thesis project. The copyright of the thesis itself belongs to the student Morgane Magnier.  

**Rights and Intellectual Property**:  
- Vattenfall has the right to use the findings, methods, and conclusions of this thesis in its operations.  
- Any material generated within the framework of this thesis that is subject to intellectual property protection (e.g., source code, computer program, design, or invention) belongs to Vattenfall, unless otherwise agreed in writing.  

Permission is granted to view, copy, and share this notebook for **educational or personal purposes only**, provided that this notice is included in all copies.  

---

In [1]:
import ee, eemont, geemap

try:
        ee.Initialize()
except Exception as e:
        ee.Authenticate()
        ee.Initialize()

import sys
sys.path.append('../preprocessing/clouds')
import landsat_preprocessing
import s2_preprocessing

import sys
sys.path.append('../wetlands_detection')
import wetlands_unsupervised_clustering

# Get collections

In [138]:
roi = ee.Geometry.Polygon([[[17.204933,60.402663],[17.204933,60.455525],[17.2645,60.455525],[17.2645,60.402663],[17.204933,60.402663]]])

In [139]:
l7 = landsat_preprocessing.get_l7_cloud_free_col(roi, ['NDVI', 'NDMI', 'NDBI'], 0.3).filter(ee.Filter.calendarRange(6, 8, 'month'))
l8 = landsat_preprocessing.get_l8_cloud_free_col(roi, ['NDVI', 'NDMI', 'NDBI'], 0.3).filter(ee.Filter.calendarRange(6, 8, 'month'))
s2 = s2_preprocessing.get_s2_cloud_free_col(roi, 0.3,['NDVI', 'NDBI', 'NDMI']).filter(ee.Filter.calendarRange(6, 8, 'month'))

l7 = l7.spectralIndices('water')
l8 = l8.spectralIndices('water')
s2 = s2.spectralIndices('water')

In [140]:
def filter_col(col, roi, band, thresh):
    
    col = col.map(lambda image : image.clip(roi))

    def count_pixels(image,roi): 
        pixel_count = image.select(band).reduceRegion(
            reducer= ee.Reducer.count(),
            geometry=roi,
            scale=10,
            maxPixels=1e9
        ).get(band)
        return image.set('pixel_count', pixel_count)

    nb_pixels_ts = col.map(lambda image: count_pixels(image, roi))

    # Get the image with the maximum pixel count
    max_pixel_count_image = nb_pixels_ts.sort('pixel_count', False).first()
    ref_img_pixel_count = max_pixel_count_image.get('pixel_count').getInfo()
    pixel_count_threshold = ref_img_pixel_count * thresh

    # Filter the collection based on the pixel count threshold
    filtered_col = nb_pixels_ts.filter(ee.Filter.gte('pixel_count', pixel_count_threshold))

    return filtered_col

def create_annual_composites(collection, roi, band):
    def add_year(image):
        date = ee.Date(image.get('system:time_start'))
        year = date.get('year')
        return image.set('year', year)
    
    # Ajouter la propriété 'year' à chaque image
    collection = collection.map(add_year)
    
    # Obtenir la liste des années uniques
    years = ee.List(collection.aggregate_array('year')).distinct().sort()
    
    # Fonction pour créer les composites annuels
    def composite_year(year):
        year = ee.Number(year)
        filtered = collection.filter(ee.Filter.calendarRange(year, year, 'year'))
        composite = filtered.median()
        return composite.set('year', year).set('system:time_start', ee.Date.fromYMD(year, 1, 1))

    # Créer les composites pour chaque année
    composites = years.map(lambda year: composite_year(year))

    filter_non_empty_bands(collection, roi, band)
    # Retourner la collection de composites
    return ee.ImageCollection(composites)


def filter_non_empty_bands(collection, roi, band):
    def has_bands(image):
        # Vérifier si l'image a des bandes en comptant les noms des bandes
        band_count = image.bandNames().size()
        return image.set('band_count', band_count)
    
    # Ajouter la propriété 'has_bands' à chaque image
    collection_with_band_info = collection.map(has_bands)
    
    # Filtrer les images qui contiennent au moins une bande
    filtered_collection = collection_with_band_info.filter(ee.Filter.gt('band_count', 0))
    filter_col(filtered_collection, roi, band, 0.99)
    
    return filtered_collection

In [141]:
l7_composites = create_annual_composites(l7, roi, 'SR_B1')
l8_composites = create_annual_composites(l8, roi, 'SR_B1')
s2_composites = create_annual_composites(s2,roi, 'B1')


l8_rs = l8_composites
l7_rs = l7_composites#.map(lambda i: i.resample('bicubic'))
s2_rs = s2_composites#.map(lambda i: i.resample('bicubic'))
    
display(l7_rs)

# Display

## NDVI density

In [ ]:
l8_img = l8_rs.first()#filter(ee.Filter.eq('year',2020)).first()

hn_min = 0.05
hn_max = 0.35
hn = 0.05

ndvi_contours = geemap.create_contours(l8_img.select('NDVI'), hn_min, hn_max, hn, region=None)

l8_rgbVis = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'min': 0.0,'max': 0.1}

color = ['FFFFFF', 'CE7E45'
         , 'DF923D', '99B718',
         '74A901', '66A000', '529400', '3E8601', 
         '207401', '056201', '004C00', '023B01', 
         '012E01', '011D01', '011301']

l8_ndviVis = {"min":0, "max":0.5, 'palette':color}

def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

density_vis = {
  'min': hn_min,
  'max': hn_max,
  'palette': ['ff0000','ffff00','ffffff']}

# initialize our map
m = geemap.Map()
m.centerObject(roi, 14)
m.addLayer(l8_img.select('NDVI'), l8_ndviVis, "NDVI")
#m.addLayer(apply_scale_factors(l8_img), l8_rgbVis, "True Color")
m.addLayer(ndvi_contours, density_vis, 'contours', True)
#m.add_colorbar(density_vis, label="NDVI density", layer_name="NDVI_density", orientation="vertical", transparent_bg=True,)
m.add_colorbar(l8_ndviVis, label="NDVI", layer_name="NDVI", orientation="horizontal")
#m

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
s2_img = s2_rs.filter(ee.Filter.eq('year',2023)).first()

hn_min = 0.4
hn_max = 0.8
hn = 0.1

density_vis = {
  'min': hn_min,
  'max': hn_max,
  'palette': ['0000ff','00ffff','ffff00','ff0000','ffffff']}

s2_rgbVis = {'bands': ['B4', 'B3', 'B2'],'min': 0.0,'max': 0.1}

color = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']

s2_ndviVis = {"min":0, "max":1, 'palette':color}

# initialize our map
ma = geemap.Map()
ma.centerObject(roi, 14)
ma.addLayer(s2_img.divide(10000), s2_rgbVis, "rgbVis")
ma.addLayer(s2_img.select('NDVI'), s2_ndviVis, "NDVI")
ndvi_contours = geemap.create_contours(s2_img.select('NDVI'), hn_min, hn_max, hn, region=None)
ma.addLayer(ndvi_contours, {'palette': ['0000ff', '00ffff', 'ffff00', 'ff0000', 'ffffff']}, 'contours', True)
ma.add_colorbar(density_vis, label="NDVI density", layer_name="NDVI", orientation="vertical", transparent_bg=True,)
ma.add_colorbar(s2_ndviVis, label="NDVI", layer_name="NDVI", orientation="horizontal", transparent_bg=True,)
#ma

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
l7_img = l7_rs.filter(ee.Filter.eq('year',2023)).first()

hn_min = 0.0
hn_max = 1
hn = 0.1

ndvi_contours = geemap.create_contours(l7_img.select('NDVI'), hn_min, hn_max, hn, region=None)

l7_rgbVis = {'bands': ['SR_B3', 'SR_B2', 'SR_B1'],'min': 0.0,'max': 0.1}

color = ['FFFFFF', 'CE7E45', 'DF923D', '99B718',
         '74A901', '66A000', '529400', '3E8601', 
         '207401', '056201', '004C00', '023B01', 
         '012E01', '011D01', '011301']

l7_ndviVis = {"min":0, "max":0.5, 'palette':color}

def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

density_vis = {
  'min': hn_min,
  'max': hn_max,
   'palette': ['ff0000','ffff00','ffffff']}


# initialize our map
m = geemap.Map()
m.centerObject(roi, 14)
m.addLayer(l7_img.select('NDVI'), l7_ndviVis, "NDVI")
m.addLayer(apply_scale_factors(l7_img), l7_rgbVis, "True Color")
m.addLayer(ndvi_contours, density_vis, 'contours', True)
m.add_colorbar(density_vis, label="NDVI density", layer_name="NDVI_density", orientation="vertical")
m.add_colorbar(l7_ndviVis, label="NDVI", layer_name="NDVI", orientation="horizontal")
m

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

## NDMI density

In [ ]:
l8_img = l8_rs.filter(ee.Filter.eq('year',2020)).first()

hn_min = 0
hn_max = 0.5
hn = 0.05

ndmi_contours = geemap.create_contours(l8_img.select('NDMI'), hn_min, hn_max, hn, region=None)

l8_rgbVis = {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'min': 0.0,'max': 0.1}

color = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']

l8_ndmiVis = {"min":0, "max":1, 'palette':color}

def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

density_vis = {
  'min': hn_min,
  'max': hn_max,
  'palette': ['0000ff','00ffff','ffff00','ff0000','ffffff']}


# initialize our map
m = geemap.Map()
m.centerObject(roi, 14)
m.addLayer(l8_img.select('NDMI'), l8_ndmiVis, "NDMI")
#m.addLayer(apply_scale_factors(l8_img), l8_rgbVis, "True Color")
m.addLayer(ndmi_contours, density_vis, 'contours', True)
m.add_colorbar(density_vis, label="NDMI density", layer_name="NDVI_density", orientation="vertical", transparent_bg=True,)
m.add_colorbar(l8_ndmiVis, label="NDMI", layer_name="NDVI", orientation="horizontal", transparent_bg=True,)
m

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
import geemap.chart as chart

samples = s2_img.sample(
        numPixels=10000,
        #classBand='label', 
        scale=30, 
        region = roi
    )

property = 'NDVI'

options = {
    "title": "Histogramm of NDVI band from S2 2023 summer composite",
    "xlabel": "NDVI band value",
    "ylabel": "Pixel count",
    "colors": ["#1d6b99"],
}

chart.feature_histogram(samples, property, **options)

In [ ]:
s2_img = s2_rs.filter(ee.Filter.eq('year',2023)).first()

hn_min = 0
hn_max = 1
hn = 0.1

ndmi_contours = geemap.create_contours(l8_img.select('NDMI'), hn_min, hn_max, hn, region=None)

density_vis = {
  'min': hn_min,
  'max': hn_max,
  'palette': ['0000ff','00ffff','ffff00','ff0000','ffffff']}

s2_rgbVis = {'bands': ['B4', 'B3', 'B2'],'min': 0.0,'max': 0.1}

color = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']

s2_ndmiVis = {"min":0, "max":0.5, 'palette':color}

def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

# initialize our map
m = geemap.Map()
m.centerObject(roi, 14)
m.addLayer(s2_img.select('NDMI'), s2_ndmiVis, "NDMI")
#m.addLayer(s2_img.divide(10000), s2_rgbVis, "True Color")
m.addLayer(ndmi_contours, density_vis, 'contours', True)
m.add_colorbar(density_vis, label="NDMI density", layer_name="NDVI_density", orientation="vertical", transparent_bg=True,)
m.add_colorbar(s2_ndmiVis, label="NDMI", layer_name="NDVI", orientation="horizontal", transparent_bg=True,)
m

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

# Test clustering

In [ ]:
l7_img = l7_composites.first()
l7_contour = geemap.create_contours(l7_img.select('NDVI'), 0, 0.4, 0.1, region=None).rename('ndvi_density')
display(l7_img)
l7_contour = l7_contour.addBands(l7_img.select('NDVI'))
display(l7_contour)

In [ ]:
l7_train = l7_img.sample(
        region = roi,
        scale = 30,
        numPixels = 10e9,
        geometries=True
        )

In [ ]:
kmeans_l7 = ee.Clusterer.wekaKMeans(nClusters = 4, init = 2, distanceFunction = 'Euclidian', maxIterations = 500).train(l7_train)
l7_cluster = l7_img.cluster(kmeans_l7)
display(l7_img, l7_cluster)

In [ ]:
# Visualize
nClusters=5
import geemap
m = geemap.Map()
# Display the RGB cluster means.
kmeans_vis = {'min': 0, 'max': nClusters - 1, 'palette': [ '#1E90FF', '#BDB76B', '#BC8F8F', '#FF4500', '#228B22']}
m.centerObject(roi,14)
m.addLayer(l7_cluster.visualize(**kmeans_vis).clip(roi), {}, 'Unsupervised K-means Classification', True, 1)
m

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

# Time series

## Calculate density and clustering 

### Remove water

In [142]:
def clustering(image,roi,scale): 
    
    bands = [
    "ANDWI",
    "AWEInsh",
    "AWEIsh",
    "LSWI",
    "MBWI",
    "MLSWI26",
    "MLSWI27",
    "MNDWI",
    "MuWIR",
    "NDPonI",
    "NDTI",
    "NDVIMNDWI",
    "NDWI",
    "NDWIns",
    "NWI",
    "SWM",
    "WI1",
    "WI2",
    "WI2015",
    "WRI"
]
    training_samples = image.select(bands).sample(region=roi, scale=scale, numPixels=15000, geometries=True)
    
    clusterer = ee.Clusterer.wekaKMeans(nClusters = 2, distanceFunction="Euclidean", maxIterations=75).train(training_samples)

    cluster = image.addBands(image.cluster(clusterer).rename('cluster'))

    return cluster

def water_classification(image, roi,scale):
    
    image = clustering(image, roi, scale)

    cluster_1 = image.updateMask(image.select('cluster').eq(1))
    cluster_2 = image.updateMask(image.select('cluster').eq(0))
    
    mean_1 = cluster_1.select('MNDWI').reduceRegion(ee.Reducer.mean(),geometry = roi, scale = scale)
    mean_2 = cluster_2.select('MNDWI').reduceRegion(ee.Reducer.mean(),geometry = roi, scale = scale)
    
    mean_1_value = mean_1.getNumber('MNDWI')
    mean_2_value = mean_2.getNumber('MNDWI')
    
    water = ee.Image(ee.Algorithms.If(mean_1_value.gt(mean_2_value), image.select('cluster').eq(0), image.select('cluster').eq(1)))

    return water

In [143]:
import geemap.chart as chart
img = s2_composites.median()
img = img.updateMask(water_classification(img, roi, 10))


In [103]:

samples = img.sample(
        numPixels=10000,
        #classBand='label', 
        scale=10, 
        region = roi
    )

property = 'NDVI'


In [104]:
options = {
    "title": "Histogramm of NDVI cluster from S2 summer composite 2017 to 2024",
    "xlabel": "NDVI band value",
    "ylabel": "Pixel count",
    "colors": ["#1d6b99"],
}

chart.feature_histogram(samples, property, **options)

In [144]:
def define_density_parameters(col, roi, scale): 
    img = col.median()
    mask = water_classification(img, roi,scale)
    stats = img.updateMask(mask).select('NDVI').reduceRegion(
        reducer=ee.Reducer.mean().combine(
            reducer2=ee.Reducer.stdDev(), 
            sharedInputs=True
        ),
        geometry=roi,
        scale=30,
        maxPixels=1e9
    )
    
    ndvi_mean = ee.Number(stats.get('NDVI_mean'))
    ndvi_std = ee.Number(stats.get('NDVI_stdDev'))
    
    # Calculer les valeurs min et max basées sur la moyenne +/- 2*stdDev
    ndvi_min = ndvi_mean.subtract(ndvi_std.multiply(3))
    ndvi_max = ndvi_mean.add(ndvi_std.multiply(3))
    
    # Calculer le pas basé sur la déviation standard
    step = ndvi_std
    nClusters = ndvi_max.subtract(ndvi_min).divide(ndvi_std).floor().int()
    return mask, ndvi_min, ndvi_max, step, nClusters

In [145]:
mask_l7, ndvi_min_l7, ndvi_max_l7, step_l7, nClusters_l7 = define_density_parameters(l7_composites, roi, 30)
mask_l8, ndvi_min_l8, ndvi_max_l8, step_l8, nClusters_l8 = define_density_parameters(l8_composites, roi,30)
mask_s2, ndvi_min_s2, ndvi_max_s2, step_s2, nClusters_s2 = define_density_parameters(s2_composites, roi, 10)

In [16]:
display(ndvi_min_l8, ndvi_max_l8, step_l8, nClusters_l8)

In [146]:
def create_contours_based_on_std(image, roi, scale, mask, ndvi_min, ndvi_max, step, nClusters):
    # Calculer la moyenne et la déviation standard du NDVI
    # Créer les contours en utilisant ces steps
    #contours = geemap.create_contours(image.select('NDVI'), 0.2, 0.5, 0.1)

    contours = geemap.create_contours(image.select('NDVI'), ee.Number(ndvi_min), ee.Number(ndvi_max), ee.Number(step))

    training = contours.sample(
        region = roi,
        scale = scale,
        numPixels = 10e9,
        geometries=True
        )
    
    kmeans = ee.Clusterer.wekaKMeans(nClusters=4, init=2, distanceFunction='Euclidian', maxIterations=500).train(training)
    clusters_image = image.updateMask(mask).cluster(kmeans).copyProperties(image,['year'])
    
    return image.select('NDVI').addBands(clusters_image)

In [167]:
min_water_date = ee.Date('2022-09-25')
max_water_date = ee.Date('2018-05-09')

wetlands = wetlands_unsupervised_clustering.getWetlandsS2(roi, min_water_date, max_water_date)
wetlands_contours = wetlands.subtract(wetlands.focal_min(radius=2)).selfMask()

In [147]:
l7_clustered = l7_composites.map(lambda image : create_contours_based_on_std(image,roi, 30,mask_l7, ndvi_min_l7, ndvi_max_l7, step_l7, nClusters_l7))
l8_clustered = l8_composites.map(lambda image : create_contours_based_on_std(image,roi, 30,mask_l8, ndvi_min_l8, ndvi_max_l8, step_l8, nClusters_l8))
s2_clustered = s2_composites.map(lambda image : create_contours_based_on_std(image,roi, 10,mask_s2, ndvi_min_s2, ndvi_max_s2, step_s2, nClusters_s2))

display(l8_clustered)

In [158]:
# Visualize
img = l7_composites.first() #filter(ee.Filter.eq('year',2023))
m = geemap.Map()
l8_rgbVis = {'bands': ['SR_B3', 'SR_B2', 'SR_B1'],'min': 0.0,'max': 0.1}
# Ajouter la légende à la carte
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

m.centerObject(roi,14)
m.addLayer(apply_scale_factors(img), l8_rgbVis, 's2')
m



Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

In [114]:
# Visualize
img = l8_clustered.filter(ee.Filter.eq('year',2023)).first() 
m = geemap.Map()

kmeans_vis_6 = {
    'min': 0,
    'max': 4,
    'palette': [
        #'#DAF7A6',  # Vert pastel
        #'#900C3F',  # Bordeaux
        '#8B8682',  # Mauve gris
        '#C70039',  # Rouge carmin
        '#FF5733',  # Rouge vif
        '#FFC300',  # Jaune éclatant
        #'#581845'   # Violet foncé
    ]
}

# Ajouter la légende à la carte
m.centerObject(roi,14)
m.addLayer(img.select('cluster').visualize(**kmeans_vis_6).clip(roi), {}, 'Unsupervised K-means Classification', True, 1)
m

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

In [88]:
legend_dict = {
    'Cluster 0 (Min NDVI values)': '#DAF7A6',  # Vert pastel
    'Cluster 1': '#FF5733',  # Rouge vif
    'Cluster 2': '#C70039',  # Rouge carmin
    'Cluster 3': '#8B8682',  # Mauve gris
    'Cluster 5 (Max NDVI values)': '#FFC300',  # Jaune éclatant
    #'Cluster 4': '#900C3F',  # Bordeaux
}

m.add_legend(legend_title="K-means Clusters", legend_dict=legend_dict)

## Clusters identificatio

In [148]:
def identify_clusters(image, roi):
    def calcul_ndvi_median(i,image):
        ndvi_median = image.select('NDVI').updateMask(image.select('cluster').eq(i)).reduceRegion(
            reducer=ee.Reducer.median(),
            geometry=roi,
            scale=30,
            maxPixels=1e9
        )
        
        # Renvoyer simplement la médiane sous forme de dictionnaire ou de valeur
        return ee.Feature(None, ndvi_median).set({'numCluster' : i})  # Retourne la valeur de la médiane du NDVI

    # Appliquer la fonction pour chaque cluster en utilisant map
    ndvi_median_values = ee.List.sequence(0, 5).map(lambda i: calcul_ndvi_median(ee.Number(i), image))
    ndvi_median_values = ee.FeatureCollection(ndvi_median_values)
    ndvi_median_values = ndvi_median_values.sort('NDVI')

    ndvi_median_values_ts = ee.data.computeFeatures({
        'expression': ndvi_median_values,
        'fileFormat': 'PANDAS_DATAFRAME'
        })
    #display(ndvi_median_values_ts)

    def reassign_clusters(image, fc):
        clusters = image.select('cluster')
        ndvi = image.select('NDVI')

        # Correction des labels de cluster
        cluster_labels = clusters \
            .where(clusters.eq(int(fc.iloc[0]['numCluster'])), 0) \
            .where(clusters.eq(int(fc.iloc[1]['numCluster'])), 1) \
            .where(clusters.eq(int(fc.iloc[2]['numCluster'])), 2) \
            .where(clusters.eq(int(fc.iloc[3]['numCluster'])), 3) \
            .where(clusters.eq(int(fc.iloc[4]['numCluster'])), 4) \
            .where(clusters.eq(int(fc.iloc[5]['numCluster'])), 5)
        #display(cluster_labels)
        # Correction des valeurs NDVI
        ndvi_image = ndvi \
            .where(clusters.eq(int(fc.iloc[0]['numCluster'])), float(fc.iloc[0]['NDVI'])) \
            .where(clusters.eq(int(fc.iloc[1]['numCluster'])), float(fc.iloc[1]['NDVI'])) \
            .where(clusters.eq(int(fc.iloc[2]['numCluster'])), float(fc.iloc[2]['NDVI'])) \
            .where(clusters.eq(int(fc.iloc[3]['numCluster'])), float(fc.iloc[3]['NDVI'])) \
            .where(clusters.eq(int(fc.iloc[4]['numCluster'])), float(fc.iloc[4]['NDVI'])) \
            .where(clusters.eq(int(fc.iloc[5]['numCluster'])), float(fc.iloc[5]['NDVI']))
        #display(ndvi)
        # Combinaison des résultats
        return cluster_labels#.rename('cluster_labels').addBands(ndvi_image.rename('NDVI_median'))

# fc est supposé être un DataFrame pandas avec les colonnes 'numCluster' et 'NDVI'

    fc = ndvi_median_values_ts
    ranked_clusters = reassign_clusters(image, fc)

    return ranked_clusters


In [178]:
img_1 = s2_clustered.filter(ee.Filter.eq('year',2017)).first()
img_2 = s2_clustered.filter(ee.Filter.eq('year',2023)).first()

In [179]:
display(img_1, img_2)

In [180]:
img = s2_composites.filter(ee.Filter.eq('year',2017)).first()
img_1 = identify_clusters(img_1, roi)
img_2 = identify_clusters(img_2, roi)

In [181]:
display(img_1, img_2)

In [124]:
import geemap.chart as chart

samples = img_1.sample(
        numPixels=1000,
        #classBand='label', 
        scale=30, 
        region = roi
    )

property = 'NDVI'

options = {
    "title": "Histogramm of NDVI from L7 summer composite",
    "xlabel": "NDVI band value",
    "ylabel": "Pixel count",
    "colors": ["#1d6b99"],
}

chart.feature_histogram(samples, property, **options)

In [182]:
kmeans_vis_6 = {
    'min': 0,
    'max': 5,
    'palette': [
        '#FF5733',  # Rouge vif
        '#FFC300',  # Jaune éclatant
        '#DAF7A6',  # Vert pastel
        '#900C3F',  # Bordeaux
        '#C70039',  # Rouge carmin
        '#581845'   # Violet foncé
    ]
}

m = geemap.Map()
m.centerObject(roi,14)
m.addLayer(img_1.select('cluster').visualize(**kmeans_vis_6).clip(roi), {}, 'Unsupervised K-means Classification_2')
m.addLayer(img_2.select('cluster').visualize(**kmeans_vis_6).clip(roi), {}, 'Unsupervised K-means Classification')
# Définir la légende
legend_dict = {
    'Cluster 1': '#FF5733',  # Rouge vif
    'Cluster 2': '#FFC300',  # Jaune éclatant
    'Cluster 3': '#DAF7A6',  # Vert pastel
    'Cluster 4': '#900C3F',  # Bordeaux
    'Cluster 5': '#C70039',  # Rouge carmin
    'Cluster 6': '#581845'   # Violet foncé
}
m.addLayer(wetlands_contours, {'palette': ['blue']}, 'Wetlands Contours')

# Ajouter la légende à la carte
m.add_legend(legend_title="K-means Clusters", legend_dict=legend_dict)
m

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…

In [164]:

m.addLayer(wetlands_contours,{'palette': ['blue']}, 'wetlands')

In [183]:
# Calculer la différence entre les deux images
difference = img_2.subtract(img_1)

# Détecter les changements positifs (clusters plus grands)
positive_change = difference.gt(0).multiply(difference)

# Détecter les changements négatifs (clusters plus petits)
negative_change = difference.lt(0).multiply(difference.abs())

positive_change = positive_change.updateMask(positive_change.neq(0))
negative_change = negative_change.updateMask(negative_change.neq(0))

In [185]:
# Appliquer une palette de couleurs en fonction du degré de changement
positive_palette = ['e5f5e0', 'a1d99b', '31a354']  # Vert clair à foncé
negative_palette = ['fee0d2', 'fc9272', 'de2d26']  # Orange clair à rouge foncé

# Visualiser les changements positifs avec une palette verte
positive_vis_params = {
    'min': 1,  # Minimum possible changement positif
    'max': 6,  # Maximum possible changement positif
    'palette': positive_palette
}

# Visualiser les changements négatifs avec une palette rouge
negative_vis_params = {
    'min': 1,  # Minimum possible changement négatif
    'max': 6,  # Maximum possible changement négatif
    'palette': negative_palette
}

# Afficher les résultats sur une carte
Map = geemap.Map()

Map.centerObject(roi, 14)
l8_rgbVis = {'bands': ['B4', 'B3', 'B2'],'min': 0.0,'max': 0.1}

Map.addLayer(img.divide(10000),l8_rgbVis,'background')
Map.addLayer(positive_change, positive_vis_params, 'Positive Change')
Map.addLayer(negative_change, negative_vis_params, 'Negative Change')

Map.add_colorbar(vis_params=positive_vis_params, label='Positive Change Degree', orientation='horizontal', position='bottomleft')
Map.add_colorbar(vis_params=negative_vis_params, label='Negative Change Degree', orientation='horizontal', position='bottomright')
Map.addLayer(wetlands_contours, {'palette': ['blue']}, 'Wetlands Contours')

Map

Map(center=[60.42909015971474, 17.23471649999347], controls=(WidgetControl(options=['position', 'transparent_b…